# SNEWPY Demo for PyHEP 2022

## Installation
If you’re running this in Binder: Great! Everything should just work.

Otherwise, use `pip install snewpy==1.3b1` to install snewpy and `git clone https://github.com/SNOwGLoBES/snowglobes.git` to download SNOwGLoBES, which we will need in the second half of this notebook to estimate event rates in realistic detectors.

## Get Sample Data Files

Let’s download data files containing neutrino fluxes predicted by a few different supernova models.

In [ ]:
import snewpy
from snewpy.models.ccsn import Nakazato_2013

import os


snewpy.get_models("Nakazato_2013")
os.path.isdir('SNEWPY_models/Nakazato_2013/')
model = Nakazato_2013('SNEWPY_models/Nakazato_2013/nakazato-shen-z0.004-t_rev100ms-s20.0.fits')
model